In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
sales = pd.read_csv('Sales.csv', sep=",", encoding="ISO-8859-1", header=0)

In [ ]:
sales.head()

In [ ]:
outlet_df = pd.read_csv('Outlet Info.csv', sep=",", encoding="ISO-8859-1", header=0)
outlet_df.head()

In [ ]:
mapping = pd.read_csv("Distributor Outlet Mapping.csv")

In [ ]:
mapping.head()

In [ ]:
sales.info()

In [ ]:
outlet_df.info()

In [ ]:
sales[sales["outlet_id"] == "outlet_code_1"]

In [ ]:
merged_df = pd.merge(sales, outlet_df, on="outlet_id")

In [ ]:
merged_df.info()

In [ ]:
merged_df.to_csv("output_file.csv",index=False, encoding="ISO-8859-1")

In [ ]:
df = pd.read_csv("output_file.csv")

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

filtered_entries = ~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR)))
clean_data = df[filtered_entries]
clean_data

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.nunique()

In [ ]:
df["expected_rainfall_num"] = pd.to_numeric(df["expected_rainfall"].str.replace("mm", ""), errors="coerce")

In [ ]:
df.corr()

In [ ]:
def clean_freezer(df: pd.DataFrame) -> pd.DataFrame:
    unique_values = np.unique(df["freezer_status"])
    values = np.array(unique_values, dtype=str)

    replacement_dict = {value: 'freezers available' for value in values if value.strip() == 'freezers available'}
    replacement_dict_2 = {'freezers available': 1, ' nofreezers available': 0}

    df["freezer_status"] = df["freezer_status"].replace(replacement_dict, regex=True)

    df["freezer_status"] = df["freezer_status"].replace(replacement_dict_2)

    return df

df = clean_freezer(df)

In [ ]:
def clean_outlet_region(df: pd.DataFrame) -> pd.DataFrame:
    encoded = pd.get_dummies(df['region'], prefix='region')
    df = pd.concat([df, encoded], axis=1)
    df = df.drop(columns=['region'])
    df = df.replace({True: 1, False: 0})

    return df

df = clean_outlet_region(df)

In [ ]:
def convert_to_date(df: pd.DataFrame):

    df["transaction_time"] = pd.to_datetime(df["transaction_time"], format='%Y-%m-%d %H:%M', errors='coerce')

    # If you want to extract the week number
    df["transaction_time"] = df["transaction_time"].dt.isocalendar().week.astype("int")

    return df

df = convert_to_date(df)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.nunique()

In [ ]:
def sort_dataframe(df, column_name)-> pd.DataFrame:
    sorted_df = df.sort_values(by=[column_name])
    return sorted_df

df = sort_dataframe(df,"transaction_time")

In [ ]:
df.head()

In [ ]:
def drop_columns(df: pd.DataFrame, columns_to_drop):
    # Create a new DataFrame without the specified columns
    new_df = df.drop(columns=columns_to_drop, errors='ignore')
    return new_df

columns_to_drop = ["outlet_id", "expected_rainfall", "sku_name"]

cleaned_df = drop_columns(df, columns_to_drop)

In [ ]:
cleaned_df.to_csv("cleaned_data.csv",index=False, encoding="ISO-8859-1")


In [ ]:
df.to_csv("without_drop_column.csv",index=False, encoding="ISO-8859-1")


In [ ]:
df = pd.read_csv("without_drop_column.csv")

In [ ]:
df.info()

In [ ]:
def clear_outlet_code(df: pd.DataFrame) -> pd.DataFrame:
    df['outlet_id_num'] = df['outlet_id'].str.extract('(\d+)').astype(int)
    return df

df = clear_outlet_code(df)

In [ ]:
df.info()

In [ ]:
grouped_df = df.groupby(["outlet_id", "transaction_time"]).agg({"sales_quantity": "sum","expected_rainfall_num":"mean","freezer_status":"mean","outlet_size":"mean","population_density":"mean","number_of_skus_sold":"mean","average_household_income":"mean","region_outstation":"mean","region_upcountry":"mean","region_western":"mean","outlet_id_num":"mean"}).reset_index()
grouped_df.head()

In [ ]:
grouped_df.shape

In [ ]:
df = grouped_df

In [ ]:
grouped_df.tail()

In [ ]:
df["outlet_id"].value_counts()

In [ ]:
def sort_dataframe(df, column_name, second_column)-> pd.DataFrame:
    sorted_df = df.sort_values(by=[column_name,second_column])
    return sorted_df

df = sort_dataframe(df, 'transaction_time','outlet_id_num')
df.head()

In [ ]:
df["outlet_id_num"].unique()

In [ ]:
df["outlet_id"].value_counts()

In [ ]:
df["transaction_time"].value_counts()

In [ ]:
df.to_csv("final_data.csv", index=False,encoding="ISO-8859-1")


In [ ]:
df = df[df['transaction_time'] < 22]
df.head()

In [ ]:
df["transaction_time"].value_counts()

In [ ]:
train_df = df[df['transaction_time'] < 17]
train_df.shape

In [ ]:
test_df = df[df['transaction_time'] > 18]
test_df.shape

In [ ]:
val_df = df[(df['transaction_time'] >= 17) & (df['transaction_time'] <= 18)]
val_df.shape

In [ ]:
def get_target_and_training_data(df: pd.DataFrame):
    Y = df['sales_quantity']
    X = df.drop(columns=['sales_quantity',"outlet_id","number_of_skus_sold"])
    return Y, X

train_Y, train_X = get_target_and_training_data(train_df)
validation_Y, validation_X = get_target_and_training_data(val_df)
test_Y, test_X = get_target_and_training_data(test_df)

In [ ]:
X_train = train_X.values.reshape((train_X.shape[0], train_X.shape[1], 1))
X_valid = validation_X.values.reshape((validation_X.shape[0], validation_X.shape[1], 1))

print("Train set reshaped", X_train.shape)
print("Validation set reshaped", X_valid.shape)

In [ ]:
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
serie_size =  X_train.shape[1] # 12
n_features =  X_train.shape[2] # 1

epochs = 20
batch = 128
lr = 0.0001

lstm_model = Sequential()
lstm_model.add(LSTM(10, input_shape=(serie_size, n_features), return_sequences=True))
lstm_model.add(LSTM(6, activation='relu', return_sequences=True))
lstm_model.add(LSTM(1, activation='relu'))
lstm_model.add(Dense(10, kernel_initializer='glorot_normal', activation='relu'))
lstm_model.add(Dense(10, kernel_initializer='glorot_normal', activation='relu'))
lstm_model.add(Dense(1))
lstm_model.summary()

adam = optimizers.Adam(lr)
lstm_model.compile(loss='mse', optimizer=adam, metrics=["mape"])

In [ ]:
lstm_history = lstm_model.fit(X_train, train_Y,
                              validation_data=(X_valid, validation_Y),
                              batch_size=32,
                              epochs=20,
                              verbose=2)